# ANOVA 

(week 10 - one way ANOVA)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.power as smp
import statsmodels.stats.proportion as smprop


## 1) ANOVA: Introduction and concept

In [ ]:
# Make pandas dataframe with grouped data:
data = pd.DataFrame({
    'value':  [2.8, 3.6, 3.4, 2.3, 5.5, 6.3, 6.1, 5.7, 5.8, 8.3, 6.9, 6.1], 
    'group':  ["A", "A", "A", "A", "B", "B", "B", "B", "C", "C", "C", "C"]})
data

In [ ]:
data.plot.scatter('group', 'value')

In [ ]:
data.boxplot(by="group", showmeans=True)

The three groups have different means - indicated by the green triangles

In [ ]:
# compare visualising the three groups seperately versus all data pooled together:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
data.boxplot(by="group", ax=axs[0], showmeans=True)
data.boxplot(ax=axs[1], showmeans=True)

Questions:

Do you think there is a significant difference between the groups? (why? or why not? and what would make the difference more clear/more significant?)

Do you think the data in the three groups could each be a **random** subset of the pooled data?

In [ ]:
# lets make a random allocation of group and plot again (run this cell several times to simulate new random groups)
data["random"] = np.random.choice(data["group"], replace=False, size=len(data))

# compare visualising the three groups seperately versus all data pooled together:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
data.boxplot(by="random", ax=axs[0], showmeans=True)
data.boxplot(ax=axs[1], showmeans=True)
plt.show()

data


(back to slides)

## 2) ANOVA: Estimate parameters $\mu$, $\alpha_i$ and $\sigma^2$

### Overall mean $\mu$:

In [ ]:
# Compute the overall mean and add to dataframe:
data['overall_mean'] = data["value"].mean()
data

### $\alpha_i$ for each group:

In [ ]:
# First compute the mean within each group and add to dataframe:
data['group_mean'] = data.groupby("group")['value'].transform('mean')
data

In [ ]:
# compute the "alpha" for each group and add to dataframe:
data["alpha"] = data["group_mean"] - data["overall_mean"]
data

### MSE - the estimate of $\sigma^2$ (withIN group variance)

First we canculate SSE (sum of squared errors - withIN each group)

In [ ]:
# calculate the individual contribution to SSE and add to dataframe:
data['sse_contribution'] = (data['value']-data['group_mean'])**2
data

In [ ]:
# calculate SSE and MSE:
SSE = data["sse_contribution"].sum()
MSE = SSE / (12-3)
print([SSE, MSE])

## 3) ANOVA: The ANOVA table with Python "ols"

In [ ]:
# Make the ANOVA table:
fit = smf.ols("value ~ group", data=data).fit()
anova_table = sm.stats.anova_lm(fit)
print(anova_table)


## 4) ANOVA: F-test

OBS: The F-test is also part of the ANOVA table output (see table above)

Here we also do the calculation *manually*

In [ ]:
# We have already calculated SSE and MSE
print([SSE, MSE])

In [ ]:
# recall the data:
data

In [ ]:
# calculate SST contribution of each datapoint:
data["sst_contribution"] = (data["value"] - data["overall_mean"])**2
data

In [ ]:
# calculate SST:
SST = data["sst_contribution"].sum()
print([SST])

In [ ]:
# We know SST = SSTr + SSE
# Calculate SSTr and MSTr:
SSTr = SST - SSE
MSTr = SSTr / (3-1)
print([SSTr, MSTr])

In [ ]:
# Now we can calculate the test-statistic F = MSTr / MSE
Fobs = MSTr / MSE
print(Fobs)

In [ ]:
# compare with critical value
print(stats.f.ppf(0.95, dfn = 3-1, dfd = 12-3))

What do we conclude when comparing Fobs anf Fcrit?

In [ ]:
# From Fobs we get a p-value:
pvalue = 1 - stats.f.cdf(Fobs, dfn = 3-1, dfd = 12-3)
print(pvalue)

Does the p-value match your expectations?

In [ ]:
# compare with values in ANOVA table:
print(anova_table)

KAHOOT (5-8)

(show slide with ANOVA table during the Kahoot)

## 5) ANOVA: Model control (analysis of residuals - are the assumptions OK?)

In [ ]:
# Visual inspection of equal variance in groups:

fig, ax = plt.subplots()
data.boxplot(column="value", by="group", showmeans=True, ax=ax)
for g in data['group'].unique():
    y = data.loc[data['group'] == g, 'value']
    x = [list(data['group'].unique()).index(g) + 1] * len(y)
    plt.plot(x, y, 'bo', alpha=0.3)
plt.show()

In [ ]:
# recall how we computed the ANOVA table using Python:
# fit = smf.ols("value ~ group", data=data).fit()
# anova_table = sm.stats.anova_lm(fit)

# from the same "fit" we can get the residuals:
data["residual"] = fit.resid
data

OBS: you can check that: 

value = overall_mean + alpha + residual

or

value = group_mean + residual


Lets inspect the residuals behave as we have assumed:

In [ ]:
# Histogram

plt.hist(data["residual"])
plt.show()

What DID we assume..? Does the residuals appear to follow the distribution that we assumed?

In [ ]:
# We could also do a QQplot:
sm.qqplot(data["residual"], line='q',a=1/2)
plt.tight_layout()
plt.show()

This look fine. Indicating that the assumption about normal residuals is not violated

In [ ]:
# residuals versus fitted values (overall_mean + alpha)
data.plot.scatter("alpha", "residual")

In [ ]:
# residual versus group:
data.plot.scatter("group", "residual")

Maybe variance of residuals is a little larger in group C (we reached the same conclusion from looking at original boxplot)

KAHOOT 9